In [429]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import zipfile
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set()

sns.set_context("talk")
import re

# Ensure that Pandas shows at least 100 characters in columns
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

from pathlib import Path

# SQL
import sqlalchemy


In [430]:
from importlib import reload
import utils as utils
reload(utils)

<module 'utils' from '/scratch/public/btruong/utils.py'>

In [431]:
alameda = pd.read_csv('alameda_u'+'_v2.csv', sep='|')
display(alameda.head())
print(alameda.shape)

/usr/local/linux/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,14,17,19,22,43,48,51,62,70,73,74,79,102,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,RowID,ImportParcelID,FIPS,State,County,ValueCertDate,ExtractDate,Edition,ZVendorStndCode,AssessorParcelNumber,DupAPN,ParcelSequenceNumber,ParcelNumberTypeStndCode,RecordSourceStndCode,RecordTypeStndCode,ConfidentialRecordFlag,PropertyAddressSourceStndCode,PropertyHouseNumber,PropertyHouseNumberExt,PropertyStreetPreDirectional,PropertyStreetName,PropertyStreetSuffix,PropertyStreetPostDirectional,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyZip,OriginalPropertyFullStreetAddress,OriginalPropertyAddressLastline,PropertyBuildingNumber,PropertyZoningDescription,PropertyZoningSourceCode,CensusTract,TaxIDNumber,TaxAmount,TaxYear,TaxDelinquencyFlag,TaxDelinquencyAmount,TaxDelinquencyYear,TaxRateCodeArea,LegalLot,LegalLotStndCode,LegalOtherLot,LegalBlock,LegalSubdivisionCode,LegalSubdivisionName,LegalCondoProjectPUDDevName,LegalBuildingNumber,LegalUnit,LegalSection,LegalPhase,LegalTract,LegalDistrict,LegalMunicipality,LegalCity,LegalTownship,LegalSTRSection,LegalSTRTownship,LegalSTRRange,LegalSTRMeridian,LegalSecTwnRngMer,LegalRecordersMapReference,LegalDescription,LegalNeighborhoodSourceCode,NoOfBuildings,LotSizeAcres,LotSizeSquareFeet,LotSizeFrontageFeet,LotSizeDepthFeet,LotSizeIRR,LotSiteTopographyStndCode,LoadID,PropertyAddressMatchcode,PropertyAddressUnitDesignator,PropertyAddressUnitNumber,PropertyAddressCarrierRoute,PropertyAddressGeoCodeMatchCode,PropertyAddressLatitude,PropertyAddressLongitude,PropertyAddressCensusTractAndBlock,PropertyAddressConfidenceScore,PropertyAddressCBSACode,PropertyAddressCBSADivisionCode,PropertyAddressMatchType,PropertyAddressDPV,PropertyGeocodeQualityCode,PropertyAddressQualityCode,SubEdition,BatchID,BKFSPID,RowID2,NoOfUnits,OccupancyStatusStndCode,PropertyCountyLandUseDescription,PropertyCountyLandUseCode,PropertyLandUseStndCode,PropertyStateLandUseDescription,PropertyStateLandUseCode,BuildingOrImprovementNumber,BuildingClassStndCode,BuildingQualityStndCode,BuildingQualityStndCodeOriginal,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,BathSourceStndCode,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FirePlaceTypeStndCode,FireplaceNumber,WaterStndCode,SewerStndCode,MortgageLenderName,TimeshareStndCode,Comments,StoryTypeStndCode,merge_row,Subsidized
0,193BFB02-2AD5-E511-80C1-3863BB43AC67,8904565,6001,CA,ALAMEDA,NaN,82010,18,BKF,1-115-1,NaN,1,NaN,NaN,NaN,,NaN,745,NaN,NaN,5TH,ST,NaN,745 5TH ST,OAKLAND,CA,94607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,17-044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.021,900.0,NaN,NaN,NaN,NaN,339073271,Y,NaN,NaN,C026,Y,37.8007,-122.282,6.0014e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,330739,-1,193BFB02-2AD5-E511-80C1-3863BB43AC67,2.0,NaN,DUPLEX OR DOUBLE,2200,RI101,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1906.0,1906.0,NaN,2.0,8.0,4.0,NaN,NaN,NaN,NaN,NaN,2.0,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,N
1,0050FA03-2AD5-E511-80C1-3863BB43AC67,8909199,6001,CA,ALAMEDA,NaN,82010,18,BKF,10-798-1-5,NaN,1,NaN,NaN,NaN,,NaN,251,NaN,NaN,28TH,ST,NaN,251 28TH ST,OAKLAND,CA,94611.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,17-001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.826,36000.0,NaN,NaN,NaN,NaN,339075041,Y,NaN,NaN,C001,Y,37.8159,-122.262,6.0014e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,330739,-1,0050FA03-2AD5-E511-80C1-3863BB43AC67,201.0,NaN,"MULTI, HIGH-RISE, 7+ STORIES",7800,RI107,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1969.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,N
2,0A50FA03-2AD5-E511-80C1-3863BB43AC67,8909213,6001,CA,ALAMEDA,NaN,82010,18,BKF,10-799-20-4,NaN,1,NaN,NaN,N

(411992, 135)


In [432]:
#Clean and filter for Berkeley JCE only
berk_jc = utils.clean_city(alameda, 'BERKELEY')
print('Current number:', berk_jc.NoOfUnits.sum())

16.67% current number: 66103.0
33.33% current number: 66101.0
50.0% current number: 66100.0
66.67% current number: 70211.0
83.33% current number: 70211.0
100.0% current number: 70225.0
Current number: 70225.0


In [433]:
# Filter for not subsidized
berk_jc = berk_jc[berk_jc['Subsidized'] == 'N']

In [434]:
# Check structure 
# (number of records, number of columns)
berk_jc.shape

(26373, 135)

In [435]:
# "Two unit properties where one unit is owner-occupied"
# Filter out properties with 2 or more units and owner occupied
berk_jc = berk_jc[~(berk_jc['OccupancyStatusStndCode'].isin(['O', 'I']) & ((berk_jc.NoOfUnits == 2) | (berk_jc.PropertyLandUseStndCode == 'RI101')))]
print('Current number:', berk_jc.NoOfUnits.sum())

Current number: 64742.0


In [436]:
# "Units rented by certain institutions of higher learning to staff, students or faculty
berk_jc = berk_jc[~(berk_jc.PropertyLandUseStndCode == 'RI113')]
print('Current number:', berk_jc.NoOfUnits.sum())

Current number: 64742.0


# Rent Control

In [447]:
# Clean and filter for Berkeley RC only
berk_rc = utils.clean_city(alameda, 'BERKELEY')
print('Current number:', berk_rc.NoOfUnits.sum())

16.67% current number: 66103.0
33.33% current number: 66101.0
50.0% current number: 66100.0
66.67% current number: 70211.0
83.33% current number: 70211.0
100.0% current number: 70225.0
Current number: 70225.0


In [448]:
# Filter for not subsidized
berk_rc = berk_rc[berk_rc['Subsidized'] == 'N']
print('Current number:', berk_rc.NoOfUnits.sum())

Current number: 69802.0


In [449]:
# "Units eligible for RC:"
# "Most multi-unit properties that were built before June 1980"
# "Units eligible for RC: Single family homes with tenants who moved in prior to 1996"

# "Units partially eligible for RC: "
# "Single-family homes first re-rented on or after 1/1/1996"
# "Most condonomiums
# APPROXIMATING THOSE ABOVE^^^^^^^^
# Select buildings before 1996, this automatically include all buildings built before 1980. 
# It will also approximate buildings where people rented, re-rented, or moved in before 1996
berk_rc = berk_rc[berk_rc.YearBuilt < 1996]
print('Current number:', berk_rc.NoOfUnits.sum())

Current number: 46566.0


In [450]:
berk_rc = berk_rc[(berk_rc.PropertyLandUseStndCode.str.contains('RI') & (berk_rc.YearBuilt < 1981)) | (berk_rc.PropertyLandUseStndCode.isin(['RR106']))]
print('Current number:', berk_rc.NoOfUnits.sum())

Current number: 26277.0


In [451]:
# "Units fully exempt from RC:"
"""
Any duplex that was owner-occupied
on December 31, 1979, and currently
has an owner living in one of the
units ("Golden Duplexes")

Rental units rented by a non-profit,
accredited institution of a higher
learning (i.e., dorms)

Non-profit cooperatives
"""
berk_rc = berk_rc[~(berk_rc.PropertyLandUseStndCode.isin(['RI113', 'RR107']))]
berk_rc = berk_rc[~((berk_rc['OccupancyStatusStndCode'].isin(['O', 'I'])) & (berk_rc.PropertyLandUseStndCode == 'RI101') & (berk_rc.YearBuilt < 1980))]

In [452]:
print('Current number:', berk_rc.NoOfUnits.sum())

Current number: 23803.0


In [453]:
print('Last JC Current number:', berk_jc.NoOfUnits.sum())
# includes full and partially coverage
print('Last RC Current number:', berk_rc.NoOfUnits.sum())

Last JC Current number: 64742.0
Last RC Current number: 23803.0
